In [67]:
# To run in Colab

#from google.colab import drive
#drive.mount('/content/drive')

# Unzipping the directory
# !pip install patool
# import patoolib
# patoolib.extract_archive('/content/drive/MyDrive/Assignments/Deep Learning for Computer Vision/Project/data.zip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
# To run locally
# If we need to unzip

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
# Colab
#my_path = '/content/data/all/data.csv'

# Local
my_path = '~/Downloads/data/all/data.csv'

df = pd.read_csv(my_path)
df = df[['path', 'borough']]
train_df, test_df = train_test_split(df, random_state = 3, test_size = 0.2)
val_df, test_df = train_test_split(test_df, random_state = 3, test_size = 0.5)

In [5]:
train_df.head()

,path,borough
34833,./data/all/Brooklyn/540593097940529.jpg,Brooklyn
3363,./data/all/Queens/479092496643392.jpg,Queens
8144,./data/all/Manhattan/1674814212725568.jpg,Manhattan
13877,./data/all/Queens/3930874653663554.jpg,Queens
14423,./data/all/Queens/3051687035117658.jpg,Queens


In [44]:
for index in range(len(train_df)):
  if index == 0:
    print(train_df.iloc[index][0])
    path = df.iloc[index][0].split('.')
    path = '~/Downloads' + path[1] + '.' + path[2]
    print(path)

./data/all/Brooklyn/540593097940529.jpg
~/Downloads/data/all/Staten Island/362372405549249.jpg


In [54]:
# Defining a custom dataset
class CustomDataset(Dataset):
    def __init__(self, df):                   # The initialization function iterates over all image files and creates a list
        self.data = []
        for index in range(len(df)):                       # For each image file
          path = df.iloc[index][0].split('.')
          path = '/Users/VassilisPanousopoulos/Downloads' + path[1] + '.' + path[2]
          if index == 0:
              print(path)
          category = df.iloc[index][1]             # Manipulate the string to find the class label
          self.data.append([path, category])
        self.class_map = {"Bronx" : 0, "Brooklyn": 1,"Manhattan": 2, "Queens": 3, "Staten Island": 4}

    def __len__(self):                                # The mandatory __len__ method returns the size of the dataset
        return len(self.data)

    def __getitem__(self, idx):                       # The mandatory __getitem__ method returns an image with its label
        img_path, class_name = self.data[idx]         # for a given index
        print(img_path, class_name)
        img = cv2.imread(img_path)                    # Reading the image, resizing, finding the label and appropriate preprocessing
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor, class_id

In [55]:
train_set = CustomDataset(train_df)
val_set = CustomDataset(val_df)
test_set = CustomDataset(test_df)

/Users/VassilisPanousopoulos/Downloads/data/all/Brooklyn/540593097940529.jpg
/Users/VassilisPanousopoulos/Downloads/data/all/Manhattan/382622079686691.jpg
/Users/VassilisPanousopoulos/Downloads/data/all/Brooklyn/391667738085900.jpg


In [51]:
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
val_loader = DataLoader(val_set, batch_size=100, shuffle=False)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False)